<b>авторы: Тендай, Сергей</b>

In [1]:
import pandas as pd
import numpy as np

In [2]:
def q(x):
    return np.array([
        -0.015
        +1.023*x[0]
        +0.969*x[1]
        +0.99*x[2]
        -0.014*x[0]*x[1]
        +0.021*x[0]*x[2]
        +0.023*x[1]*x[2],
            -0.014
            -1.01*x[0]
            -0.923*x[1]
            +1.042*x[2]
            -0.021*x[0]*x[1]
            -0.029*x[0]*x[2]
            -0.023*x[1]*x[2],
                    -0.025
                     -0.989*x[0]
                     +0.965*x[1]
                     +0.979*x[2]
                     +0.017*x[0]*x[1]
                     +0.014*x[0]*x[2]
                     +0.069*x[1]*x[2]
                    ])

### критерия -> min

In [3]:
def F(x):
    return np.array([x[0]**2.0 - 1, x[1]**2.0 - 1, x[2]**2.0 - 1])

In [4]:
x = np.repeat(0.5,3)
x

array([0.5, 0.5, 0.5])

In [5]:
z = q(x)
z

array([ 1.4835 , -0.47775,  0.4775 ])

In [6]:
u = 1/z
u

array([ 0.67408156, -2.09314495,  2.09424084])

### минимизация частных критериев

In [7]:
def Fxx(x,i):
    return u[i]*q(x)[i] + 0.0001*np.dot(u,q(x))

In [8]:
from scipy.optimize import minimize, rosen

In [9]:
cons = ({'type': 'ineq', 'fun': lambda x:  -F(x)})
bnds = ((-1,1),(-1,1),(-1,1))

In [10]:
b = []
for i in range(x.shape[0]):
    b.append(minimize(Fxx, x, i, constraints=cons, bounds=bnds).x)

In [11]:
b

[array([-1., -1., -1.]), array([-1., -1.,  1.]), array([ 1., -1., -1.])]

In [12]:
Q = q(b)
Q

array([[-1.075, -2.967, -1.047],
       [ 2.992,  0.804, -0.954],
       [ 0.912, -0.88 ,  0.878]])

### матрица крайних точек критериев

In [13]:
Q
x.shape

(3,)

### оценка краев критериев

In [14]:
m = x.shape[0]
q1 = np.zeros(m)
q2 = np.zeros(m)
for j in range(0,Q.shape[0]):
    q1[j] = min(Q[j])
    q2[j] = max(Q[j])
q1,q2

(array([-2.967, -0.954, -0.88 ]), array([-1.047,  2.992,  0.912]))

### построение сети

In [15]:
def gen(**kwargs): 
    h = kwargs.get('h') # step 
    a, b = kwargs.get('borders_x') # x borders 
    m = kwargs.get('m') # criteria count 
    k = (b - a) / h # less count of point for each coordinate 
    A = int((k + 1) ** m) # count of permutation 
    L = np.zeros((A, m)) # out 
    for i in range(A): 
        r = i 
        x = np.zeros(m, dtype=int) 
        for j in range(m): 
            x[j] = r % (k + 1) 
            r = r // (k + 1)
        L[i] = x 
    L = np.array([i for i in L if 0 in i]) 
    return L

In [16]:
h = 1.0/3.0
ll = gen(h=h,borders_x = (0,1),m=x.shape[0])*h
llDf = pd.DataFrame(ll, columns=["Л1","Л2","Л3"])

batch2 = llDf.loc[18:].reset_index().drop("index", axis=1)
pd.concat([llDf.loc[:18],batch2], axis=1)

,Л1,Л2,Л3,Л1,Л2,Л3
0,0.000000,0.000000,0.000000,0.666667,0.000000,0.333333
1,0.333333,0.000000,0.000000,1.000000,0.000000,0.333333
2,0.666667,0.000000,0.000000,0.000000,0.333333,0.333333
3,1.000000,0.000000,0.000000,0.000000,0.666667,0.333333
4,0.000000,0.333333,0.000000,0.000000,1.000000,0.333333
5,0.333333,0.333333,0.000000,0.000000,0.000000,0.666667
6,0.666667,0.333333,0.000000,0.333333,0.000000,0.666667
7,1.000000,0.333333,0.000000,0.666667,0.000000,0.666667
8,0.000000,0.666667,0.000000,1.000000,0.000000,0.666667
9,0.333333,0.666667,0.000000,0.000000,0.333333,0.666667


In [17]:
def W(x):
    return (q(x) - q1)/(q2 - q1)
W(x)
        

array([2.31796875, 0.12069184, 0.75753348])

In [18]:
def fz(x,z):
    return z + 0.0001*W(x).sum()


In [19]:
z = 0
x = np.zeros(m)


In [20]:
cols=["Л1","Л2","Л3", "x1", "x2", "x3", "q1", "q2", "q3"]
rr = np.zeros((ll.shape[0],len(cols)))
for i in range(0,ll.shape[0]):
    cons = ({'type': 'ineq', 'fun': lambda x:  -F(x)},
       {'type': 'ineq', 'fun': lambda x:  -W(x) + ll[i] + z})
    dd = minimize(fz,x,z,constraints=cons, bounds=bnds).x
    rr[i] = np.concatenate([ll[i],
                    dd,
                    q(dd)])
    

df = pd.DataFrame(rr, columns=cols)
df = df.dropna()
df.drop_duplicates()

,Л1,Л2,Л3,x1,x2,x3,q1,q2,q3
0,0.000000,0.000000,0.000000,0.100537,-0.700109,-1.000000,-1.565580,-0.523050,-1.733333
1,0.333333,0.000000,0.000000,0.146548,-0.650824,-1.000000,-1.472503,-0.612019,-1.735748
2,0.666667,0.000000,0.000000,0.218669,-0.573572,-1.000000,-1.326737,-0.751666,-1.739377
3,1.000000,0.000000,0.000000,-0.029529,-0.043885,-0.970536,-1.047000,-0.956807,-0.984937
4,0.000000,0.333333,0.000000,-0.538889,-1.000000,-1.000000,-2.498511,0.361333,-1.350333
5,0.333333,0.333333,0.000000,-0.715416,-0.707937,-0.967476,-2.367464,0.307424,-0.882213
6,0.666667,0.333333,0.000000,-0.403536,-0.660003,-0.673307,-1.721737,0.277476,-0.882979
7,1.000000,0.333333,0.000000,-0.083433,-0.489257,-0.498527,-1.062072,-0.005287,-0.884570
8,0.000000,0.666667,0.000000,-1.000000,-1.000000,-1.000000,-2.967000,0.804000,-0.880000
9,0.333333,0.666667,0.000000,-0.720764,-0.833767,-0.850995,-2.381960,0.550075,-0.882113
